In [1]:
# import pandas as pd
# segGeo = pd.read_csv('/Users/rachelspinti/Documents/River_bifurcation/all_basins_unfiltered_workflow/Colorado_segGeo.csv')
# rsegGeo = pd.read_csv('/Users/rachelspinti/Documents/River_bifurcation/all_basins_unfiltered_workflow/Red_segGeo.csv')

In [2]:
# segGeo
# rsegGeo

# Joining NABD and NID

In [3]:
import geopandas as gp, pandas as pd, matplotlib as mpl
from pathlib import Path
gdrive = Path("/Volumes/GoogleDrive/My Drive/Condon_Research_Group/Research_Projects/Rachel/Research/Data/bifurcation_data_repo") #where shapefiles/csv live 
# gdrive2 = Path("/Volumes/GoogleDrive/My Drive/Condon_Research_Group/Research_Projects/Jen_Dams/Matching") #where shapefiles/csv live 

### Read in data

In [4]:
nabd_dams = gp.read_file(gdrive/"nabd/nabd_fish_barriers_2012.shp")  #read in NABD from Drive
nabd_dams = nabd_dams.drop_duplicates(subset='NIDID', keep="first")  #drop everything after first duplicate
# nabd_dams["DamID"] = range(len(nabd_dams.COMID))  #add DamID 
nabd_dams = pd.DataFrame(nabd_dams)
nabd_dams = nabd_dams[['COMID', 'NIDID', 'Norm_stor', 'Max_stor','Year_compl', 'Purposes', 'geometry', 'Dam_name']]
nid = pd.read_csv(gdrive/'other_dam_datasets/NID2019_U.csv', usecols=['NIDID', 'NORMAL_STORAGE', 'MAX_STORAGE', 'YEAR_COMPLETED', 'DAM_NAME', 'LATITUDE', 'LONGITUDE'])

### Checking things out

In [5]:
print(len(nabd_dams), len(nid))
nabd_dams = nabd_dams.sort_values('NIDID')
nabd_dams
nid

51795 91457


,DAM_NAME,NIDID,LONGITUDE,LATITUDE,YEAR_COMPLETED,MAX_STORAGE,NORMAL_STORAGE
0,COOPER LAKE,AK00001,-149.823147,60.433708,1959.0,127600.0,112000.0
1,BLUE LAKE,AK00002,-135.191700,57.063300,1961.0,266000.0,266000.0
2,SALMON CREEK,AK00003,-134.403608,58.341850,1914.0,18000.0,12000.0
3,ANNEX CREEK,AK00004,-134.126578,58.326939,1968.0,23400.0,23400.0
4,CRYSTAL LAKE,AK00005,-132.845500,56.600000,1955.0,5800.0,5200.0
...,...,...,...,...,...,...,...
91452,SHELL CREEK NO. 2,WY02601,-107.413988,44.516375,1957.0,1949.0,1949.0
91453,WASHAKIE DIKE NO. 1,WY02902,-109.006036,42.979746,1935.0,10300.0,7940.0
91454,WASHAKIE DIKE NO. 2,WY02903,-109.011705,42.979860,1935.0,10300.0,7940.0
91455,WASHAKIE DIKE NO. 3,WY02904,-109.014574,42.979238,1935.0,10300.0,7940.0


### Figuring out what is missing

In [6]:
#Merge on nabd
# nabd_by_ID = nabd_dams.merge(nid, on = 'NIDID', how = 'inner')
# nabd_by_ID

#Merge on nid
nid.NORMAL_STORAGE = nid.NORMAL_STORAGE * 1233.48   #convert units to cubic meters
nid_by_ID = nid.merge(nabd_dams, on = 'NIDID', how = 'left')
nid_by_ID['COMID'] = nid_by_ID['COMID'].fillna(0)
missing = nid_by_ID[nid_by_ID['COMID'] ==0]
print('missing #',len(missing))

large_missing = missing[missing['NORMAL_STORAGE']>=10**8]  #is it 10^8 or 10^6?
print('large missing #', len(large_missing))
large_missing.to_csv('large_dams_missing.csv')

missing # 42648
large missing # 171


## Spatially join NHD and NID

In [7]:
flowlines = gp.read_file(gdrive/"nhd/NHDFlowlines.csv",
                                usecols=['Hydroseq', 'UpHydroseq', 'DnHydroseq',
                                        'REACHCODE','LENGTHKM', 'COMID', 'WKT', 'QC_MA',
                                        'StreamOrde'])

In [1]:
nid_gdf = gp.GeoDataFrame(nid, geometry=gp.points_from_xy(nid.LONGITUDE, nid.LATITUDE))
# nid_gdf

NameError: name 'gp' is not defined

In [ ]:
nid_nhd_join = gp.sjoin(nid_gdf, flowlines, how="left", op='intersects')
nid_nhd_join 

## Junk

In [9]:
# nid.NORMAL_STORAGE = nid.NORMAL_STORAGE * 1233.48   #convert units to MCM

# for dam in range(len(nid)):
#     missing = []
#     grand_dams = []
#     if nid.loc[dam, 'NIDID'] != nabd_dams.loc[dam, 'NIDID']:
#         missing.append(nid.NIDID)
#         if nid.loc[dam, 'NORMAL_STORAGE'] > 10**8:
#             grand_dams.append(nid.NIDID)

# print(len(missing), len(grand_dams))
# missing

#Trying to join by name
# large_missing['DAM_NAME'] = large_missing['DAM_NAME'].str.title().str.strip()
# nabd_dams['Dam_name'] = nabd_dams['Dam_name'].str.title().str.strip()
# df['Region'] = df['Region'].str.title().str.strip()
# nabd_dams
# nid['DAM_NAME'] = nid['DAM_NAME'].str.title().str.strip()
# nid = nid.rename(columns={'DAM_NAME':'Dam_name'})
# by_name = nid.merge(nabd_dams, on = 'Dam_name', how = 'left')

In [10]:
lines = pd.read_csv('/Volumes/GoogleDrive/My Drive/Condon_Research_Group/Research_Projects/Rachel/Research/Data/bifurcation_data_repo/nhd/NHDFlowlines.csv')


In [12]:
x = list(lines.columns)
x

['WKT',
 'OBJECTID',
 'COMID',
 'FDATE',
 'RESOLUTION',
 'GNIS_ID',
 'GNIS_NAME',
 'LENGTHKM',
 'REACHCODE',
 'FLOWDIR',
 'WBAREACOMI',
 'FTYPE',
 'FCODE',
 'Shape_Length',
 'StreamLeve',
 'StreamOrde',
 'StreamCalc',
 'FromNode',
 'ToNode',
 'Hydroseq',
 'LevelPathI',
 'Pathlength',
 'TerminalPa',
 'ArbolateSu',
 'Divergence',
 'StartFlag',
 'TerminalFl',
 'DnLevel',
 'UpLevelPat',
 'UpHydroseq',
 'DnLevelPat',
 'DnMinorHyd',
 'DnDrainCou',
 'DnHydroseq',
 'FromMeas',
 'ToMeas',
 'RtnDiv',
 'VPUIn',
 'VPUOut',
 'AreaSqKM',
 'TotDASqKM',
 'DivDASqKM',
 'Tidal',
 'TOTMA',
 'WBAreaType',
 'PathTimeMA',
 'HWNodeSqKM',
 'MAXELEVRAW',
 'MINELEVRAW',
 'MAXELEVSMO',
 'MINELEVSMO',
 'SLOPE',
 'ELEVFIXED',
 'HWTYPE',
 'SLOPELENKM',
 'QA_MA',
 'VA_MA',
 'QC_MA',
 'VC_MA',
 'QE_MA',
 'VE_MA',
 'QA_01',
 'VA_01',
 'QC_01',
 'VC_01',
 'QE_01',
 'VE_01',
 'QA_02',
 'VA_02',
 'QC_02',
 'VC_02',
 'QE_02',
 'VE_02',
 'QA_03',
 'VA_03',
 'QC_03',
 'VC_03',
 'QE_03',
 'VE_03',
 'QA_04',
 'VA_04',
 'QC_04